## Wyzwanie 'Coding Agent'

### Cel Zadania:
Uczestnicy mają za zadanie zaprojektować i zaimplementować agenta AI, który będzie zdolny do samodzielnego pisania efektywnego kodu rozwiązującego zadane problemy algorytmiczne.

### Opis:
W ramach warsztatów uczestnicy stworzą agenta opartego o model językowy gpt-3.5, który będzie w stanie interpretować opisy problemów algorytmicznych i automatycznie generować odpowiedni kod w pythonie. Model powinien radzić sobie z różnorodnymi typami problemów, od prostych algorytmów sortowania po bardziej złożone zagadnienia takie jak algorytmy na grafach czy struktury danych.

### Wymagania:
1. Wybór i uzasadnienie wykorzystanego systemu agenta AI.
2. Implementacja agenta, który będzie testowany na zestawie standardowych problemów algorytmicznych.
3. Osiągnięcie przynajmniej 70% na benchmarku 'lvl 1'

### Ranking
Udostępnij swoje najlepsze accuracy dla zbioru testowego w arkuszu - https://docs.google.com/spreadsheets/d/1jGV7RTz4xHvFUc-8msqLn3jhUiTX6QZ8-7rv6n6z07E/edit?usp=sharing

Na koniec warsztatu proszę wrzucić swoje notebooki z rozwiązaniami do repo, katalog `rozwiazania`.

## Setup

In [ ]:
# load the data from gdrive
import gdown

GDRIVE_DATA = [("1pYt9pH0TXhVaOBOG026v441Mv4qKqwZI", "utils.py"),
                ("1fVjQ4WcRun5xaDd8vwBLEB1ZAfTBCcFm", "requirements.txt"),
                ("1OP-NYC619eZdtrQWBkRY3C0tu-FXnzas","problems_lvl_1.jsonl"),
                ("19_tVo2yD_PgpLfmwRvr13U-82I5ANqp9","problems_lvl_2.jsonl"),
                ("1BCHOoMAGGqt65fk3XITpQcqp8pyvfnKO","problems_lvl_3.jsonl")]

for file_id, zip_name in GDRIVE_DATA:
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, zip_name, quiet=False)

In [ ]:
# RUN IF YOU ARE USING COLAB
# install requirements
!pip install -r requirements.txt

In [ ]:

import openai
from tqdm import tqdm
import os
import numpy as np
import importlib
import multiprocessing
from utils import load_problems, test, benchmark, Problem, execute_program, count_tokens

problems_lvl_1 = load_problems(lvl=1)
problems_lvl_2 = load_problems(lvl=2)
problems_lvl_3 = load_problems(lvl=3)
# jeżeli nie macie doładowanego konta, proszę się zgłosić
# Póki co użyjcie tego klucza - '###'
client = openai.OpenAI(api_key="###")

In [ ]:
print(problems_lvl_3[0])

## Przykład zadania

Twój algorytm może korzystać jedynie z atrybutów problemu `statement`, `function_signature` i `example_test`. Atrybut `tests` będzie używany jedynie podczas testowania. Aby uzyskać więcej informacji zapoznaj się z klasą `Problem` oraz funkcją `test` z pliku `utils.py`

In [ ]:
def baseline(problem):
    prompt = f"{problem.statement} Start with 'def {problem.function_signature}:'"
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Solve given problem in python. Return just the code of function."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [ ]:
result = test(baseline, problems_lvl_1[0], verbose=True)

## Guideline

Zbiór `lvl_1` ma niecałe 1000 zadań. Nie mamy czasu, aby testować rozwiązań na nich wszystkich. Przyjmijmy, że na potrzeby rozwoju agentów (`train set`) będziemy używać losowo wybranych 50 zadań. Aby to osiągnąć należy ustawić wartość `seed` w funkcji benchmark na dowolną wartość całkowitą. Nasze rozwiązania będziemy testować na defaultowej wartości `seed=None`, która bierze pierwsze n zadań ze zbioru.

# Twoje Rozwiązania - Lvl 1

Zacznij od uzupełnienia definicji agentów poniżej. Następnie przetestuj swoje rozwiązania funkcją `benchmark`.

[Large Language Models are Zero-Shot Reasoners - arXiv](https://arxiv.org/pdf/2205.11916)

In [24]:
def get_prompt(problem):
  prompt = f"{problem.statement} Start with 'def {problem.function_signature}:' End with '{problem.example_test}'"
  return prompt

prompt1 = get_prompt(problems_lvl_1[0])

answer1 = """```python
def min_cost(cost, m, n):
    dp = [[0 for _ in range(n+1)] for _ in range(m+1)]

    dp[0][0] = cost[0][0]

    for i in range(1, m+1):
        dp[i][0] = dp[i-1][0] + cost[i][0]

    for j in range(1, n+1):
        dp[0][j] = dp[0][j-1] + cost[0][j]

    for i in range(1, m+1):
        for j in range(1, n+1):
            dp[i][j] = cost[i][j] + min(dp[i-1][j], dp[i][j-1])

    return dp[m][n]
```"""

In [27]:
def get_prompt(problem):
  prompt = f"{problem.statement} Start with 'def {problem.function_signature}:' End with '{problem.example_test}' to test the corectness"
  return prompt

In [ ]:
def baseline(problem):
    prompt = get_prompt(problem)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Solve given problem in python. Return just the code of function."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

test(baseline, problems_lvl_1[1], verbose=True)

## Zero-shot

In [ ]:
def coding_agent_zero_shot(problem):
    prompt = f"{problem.statement} Start with 'def {problem.function_signature}:'"
    #prompt = f"{problem.statement} Start with 'def {problem.function_signature}:' The example test is: {problem.example_test} make sure that it will work on this test"
    #prompt = f"{problem.statement} Start with 'def {problem.function_signature}:'. Make sure this assert doesn't return an error: {problem.example_test}"
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Solve given problem in python. Return just the code of function."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [ ]:
result = test(coding_agent_zero_shot, problems_lvl_1[0], verbose=True)

In [ ]:
benchmark(coding_agent_zero_shot, problems_lvl_1, n_samples=50, verbose=False, seed=44)

# Few-shot

In [ ]:
def coding_agent_few_shot(problem):
    prompt = f"{problem.statement} Start with 'def {problem.function_signature}:'"
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Solve given problem in python. Return just the code of function."},
            {"role": "user", "content": "**Task**"},
            {"role": "assistant", "content":"**Solution**"},
            {"role": "user", "content": "**Task**"},
            {"role": "assistant", "content":"**Solution**"},
            {"role": "user", "content": "**Task**"},
            {"role": "assistant", "content":"**Solution**"},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [ ]:
result = test(coding_agent_few_shot, problems_lvl_3[1], verbose=True)

In [ ]:
benchmark(coding_agent_few_shot, problems_lvl_3, n_samples=5, verbose=False, seed=44)

## Chain-of-thoughts

In [ ]:
def coding_agent_chain_of_thoughts(problem):
    prompt = f"{problem.statement} Start with 'def {problem.function_signature}:'"
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "**write instruction in here**"},
            {"role": "user", "content": "**Task**"},
            {"role": "assistant", "content":"**Solution**"},
            {"role": "user", "content": "**Task**"},
            {"role": "assistant", "content":"**Solution**"},
            {"role": "user", "content": "**Task**"},
            {"role": "assistant", "content":"**Solution**"},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [ ]:
result = test(coding_agent_chain_of_thoughts, problems_lvl_1[0], verbose=True)

In [ ]:
benchmark(coding_agent_chain_of_thoughts, problems_lvl_1, n_samples=50, verbose=False, seed=44)

# Twoje Rozwiązania - Developer & Tester

[Teaching Large Language Models to Self-Debug - arXiv](https://arxiv.org/pdf/2304.05128)

Poniżej zaimplementowany jest właśnie ten schemat. Mamy dwóch agentów - dewelopera i testera. Pierwszy stara się rozwiązać problem. Drugi wykonuje kod i komentuje rozwiązanie. Rozwiązanie zostaje zwrócone, gdy przejdzie przykładowe testy lub limit iteracji zostanie przekroczony.

## Iteracyjne ulepszanie

In [ ]:
class SelfDebuggingAgent_continuous_development:
    def __init__(self):
        self.context_tokens_limit = 16572
        self.iterations = 3
        self.tester_system_message = "**WRITE YOUR PROMPT IN HERE**"
        self.developer_system_message = "**WRITE YOUR PROMPT IN HERE**"
        self.chat_history_developer_perspective = []
        self.chat_history_tester_perspective = []
        self.problem = None
        self.solution = None

    def Tester(self):
        # it executes the proposed solution, analyse the output and return the feedback
        execution_logs, is_correct = self.execute_code(self.solution, self.problem.example_test)
        self.chat_history_tester_perspective[-1]["content"] += f"Execution logs:\n{execution_logs}"
        feedback = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.chat_history_tester_perspective
        )

        self.chat_history_developer_perspective.append({"role": "user", "content": feedback.choices[0].message.content})
        self.chat_history_tester_perspective.append({"role": "assistant", "content": feedback.choices[0].message.content})
        self.chat_history_tester_perspective.append({"role": "user", "content": ""})

        return is_correct

    def Developer(self):
        # it generates a solution for the problem
        code = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.chat_history_developer_perspective
        )
        self.solution = code.choices[0].message.content
        self.chat_history_developer_perspective.append({"role": "assistant", "content": self.solution})
        self.chat_history_tester_perspective[-1]["content"] += f"Proposed code:\n {self.solution}\n\n"


    def execute_code(self, code, tests):
        if type(tests) == str:
            program = code + "\n" + tests
        elif type(tests) == list:
            program = "\n".join([code] + tests)

        q = multiprocessing.Queue()
        process = multiprocessing.Process(target=execute_program, args=(program, q))
        process.start()
        process.join(10.0)

        if process.is_alive():
            process.terminate()
            process.join()
            is_correct, error = False, "Timeout"
        elif q.empty():
            is_correct, error = False, "Unknown"
        else:
            is_correct, exception = q.get()
            if exception is None:
                error = None
            else:
                error = "Tests failed" if isinstance(exception, AssertionError) else str(exception)

        execution_logs = f"Passed all tests? {is_correct}" + f'\nError: {error}' if error else ""
        return execution_logs, is_correct

    def count_chat_tokens(self,chat_history):
        # return sum([count_tokens(turn["content"]) for turn in chat_history])
        return sum([len(turn["content"].split())*4/3.5 for turn in chat_history])

    def initialize_chat(self,problem):
        self.chat_history_developer_perspective = [{"role": "system", "content": self.developer_system_message}]
        self.chat_history_tester_perspective = [{"role": "system", "content": self.tester_system_message}]

        # tester initialization per problem
        self.problem = problem
        self.chat_history_tester_perspective.append({"role": "user", "content": self.problem.statement + " Start with 'def " + self.problem.function_signature + ":'\n\n"})

        # developer initialization per problem
        prompt = f"{self.problem.statement} Start with 'def {self.problem.function_signature}:'"
        self.chat_history_developer_perspective.append({"role": "user", "content": prompt})

    def __call__(self, problem: Problem):
        self.initialize_chat(problem)
        self.Developer()
        for _ in range(self.iterations):
            if self.count_chat_tokens(self.chat_history_tester_perspective) > self.context_tokens_limit:
                break
            is_correct = self.Tester()
            if is_correct or self.count_chat_tokens(self.chat_history_developer_perspective) > self.context_tokens_limit:
                break
            self.Developer()
        return self.solution

In [ ]:
benchmark(SelfDebuggingAgent_continuous_development(), problems_lvl_1, n_samples=50, verbose=False, seed=44)

## Zoptymalizowana wersja

W tym przypadku model nie zapamiętuje całej historii konwersacji. Deweloper dostaje jedynie treść, poprzednie rozwiązanie i feedback of testera. Tester dostaje jedynie treść zadania oraz najnowsze wygenerowane rozwiązanie.

In [ ]:

class SelfDebuggingAgent:
    def __init__(self):
        self.context_tokens_limit = 16572
        self.iterations = 5
        self.tester_system_message = "**WRITE YOUR PROMPT IN HERE**"
        self.developer_system_message = "**WRITE YOUR PROMPT IN HERE**"
        self.chat_history_developer_perspective = []
        self.chat_history_tester_perspective = []
        self.problem = None
        self.solution = None

    def Tester(self):
        # it executes the proposed solution, analyse the output and return the feedback
        self.chat_history_tester_perspective[1] = {"role": "user", "content": self.problem.statement + " Start with 'def " + self.problem.function_signature + ":'\n\n"}
        self.chat_history_tester_perspective[1]["content"] += f"Proposed code:\n {self.solution}\n\n"
        execution_logs, is_correct = self.execute_code(self.solution, self.problem.example_test)
        self.chat_history_tester_perspective[1]["content"] += f"Execution logs:\n{execution_logs}"
        feedback = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.chat_history_tester_perspective
        )

        self.chat_history_developer_perspective[3] = {"role": "user", "content": feedback.choices[0].message.content}

        return is_correct

    def Developer(self):
        # it generates a solution for the problem
        code = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.chat_history_developer_perspective
        )

        self.solution = code.choices[0].message.content


    def execute_code(self, code, tests):
        if type(tests) == str:
            program = code + "\n" + tests
        elif type(tests) == list:
            program = "\n".join([code] + tests)

        q = multiprocessing.Queue()
        process = multiprocessing.Process(target=execute_program, args=(program, q))
        process.start()
        process.join(10.0)

        if process.is_alive():
            process.terminate()
            process.join()
            is_correct, error = False, "Timeout"
        elif q.empty():
            is_correct, error = False, "Unknown"
        else:
            is_correct, exception = q.get()
            if exception is None:
                error = None
            else:
                error = "Tests failed" if isinstance(exception, AssertionError) else str(exception)

        execution_logs = f"Passed all tests? {is_correct}" + f'\nError: {error}' if error else ""
        return execution_logs, is_correct

    def initialize_chat(self,problem):
        # tester initialization per problem
        self.chat_history_tester_perspective = [{"role": "system", "content": self.tester_system_message}]
        self.problem = problem
        self.chat_history_tester_perspective.append({"role": "user", "content": self.problem.statement + " Start with 'def " + self.problem.function_signature + ":'\n\n"})

        # developer initialization per problem
        self.chat_history_developer_perspective = [{"role": "system", "content": self.developer_system_message}]
        prompt = f"{self.problem.statement} Start with 'def {self.problem.function_signature}:'"
        self.chat_history_developer_perspective.append({"role": "user", "content": prompt})

    def __call__(self, problem: Problem):
        self.initialize_chat(problem)
        self.Developer()
        for i in range(self.iterations):
            if i == 0:
                self.chat_history_developer_perspective.append({"role":"assistant","content":self.solution})
                self.chat_history_developer_perspective.append({"role":"user","content":""})
            else:
                self.chat_history_developer_perspective[2] = {"role":"assistant","content":self.solution}

            is_correct = self.Tester()
            if is_correct:
                break
            self.Developer()
        return self.solution

In [ ]:
benchmark(SelfDebuggingAgent(), problems_lvl_1, n_samples=50, verbose=False, seed=44)

# Twoje Rozwiązania - Lvl 3

W tej części skupiamy się na jakości agenta, nie na optymalności czasowej. Użyj modelu `gpt-4o`, aby rozwiązać kilka zadań z Olimpiady Informatycznej (gimnazjalistów). Nie ma wymogów formalnych dotyczących rozwiązania. Możecie korzystać z dowolnych frameworków i gotowych rozwiązań jakie znajdziecie w internecie.

Dobrym początkiem będzie prześledzenie jak inni radzą sobie z tego typu zadaniem - https://paperswithcode.com/sota/code-generation-on-mbpp

Jako krok pośredni możesz przetestować swój algorytm na zbiorze `lvl_2`

In [ ]:
def the_coding_agent(problem: Problem):
    pass

In [ ]:
benchmark(the_coding_agent(), problems_lvl_3, n_samples=5, verbose=False)